In [4]:
from pypower.api import loadcase, runopf, ppoption
from pypower.idx_brch import *
from pypower.idx_bus import *
from pypower.idx_gen import *
from pypower.idx_cost import *
from pypower.ext2int import ext2int
import oct2py
import numpy as np
import math
import os
from tqdm import tqdm
from oct2py import Oct2Py
octave = Oct2Py()
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import pickle
import shutil
import cvxpy as cp
if shutil.which('latex') is None:
    LATEX_AVAILABLE = False
else:
    LATEX_AVAILABLE = True
if LATEX_AVAILABLE:
    plt.rcParams['font.size'] = 14
    plt.rcParams['text.usetex'] = True
    plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,amssymb}'
    
MAX_BUS = 1500 # upper limit of number of buses in cases to be considered

In [5]:
# get all cases in current directory
current_directory = os.getcwd()
all_files_and_directories = os.listdir(current_directory)
case_files = [os.path.join(current_directory, f) for f in all_files_and_directories if f.endswith('.m') and os.path.isfile(os.path.join(current_directory, f))]

cases, casenames = [], []
cases_full, casenames_full = [], []
for cf in (t:=tqdm(case_files)):
    octave.source(os.path.basename(cf))
    cname = os.path.basename(cf).split('.')[0]
    num_buses = None
    # determine number of buses in the case from its name
    for ci in cname.split('_'):
        if 'case' in ci:
            num_buses = int(''.join([chr for chr in ci.replace('case','',1) if chr.isdigit()]))
    # fitler out cases with more buses than MAX_BUS
    if num_buses <= MAX_BUS:
        # convert to internal indexing
        case_correct_idx = ext2int(loadcase(octave.feval(cname)))
        # append
        cases.append(case_correct_idx)
        casenames.append(cname)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66/66 [00:39<00:00,  1.69it/s]


In [6]:
# this_case = cases[0]

# CVXPY SWITCHES FOR ENABLING/DISABLING CONSTRAINTS
DISABLE_MODEL = False
DISABLE_FLOWLIMS = True
DISABLE_ANG = False
DISABLE_SOCR = False
DISABLE_BALANCE = False
DISABLE_VLIMS = False
DISABLE_GENLIMS = False
USE_ONE_AS_OBJ = False # use trivial objective function, i.e. 1
print(f'''Status of constraints:
        DISABLE_MODEL: {DISABLE_MODEL}
        DISABLE_FLOWLIMS: {DISABLE_FLOWLIMS}
        DISABLE_ANG: {DISABLE_ANG}
        DISABLE_SOCR: {DISABLE_SOCR}
        DISABLE_BALANCE: {DISABLE_BALANCE}
        DISABLE_VLIMS: {DISABLE_VLIMS}
        DISABLE_GENLIMS: {DISABLE_GENLIMS}
        ''')

# RECORDS FOR SOLVED AND UNSOLVED
solved, unsolved = 0,0

for cn,this_case in tqdm(zip(casenames,cases)):
    
    # LOAD DATA
    # baseMVA
    baseMVA = this_case['baseMVA']
    # bus data
    bus_list = this_case['bus'][:,BUS_I].astype(int)
    bus_vmin, bus_vmax = this_case['bus'][:,VMIN], this_case['bus'][:,VMAX]
    bus_pd, bus_qd = this_case['bus'][:,PD], this_case['bus'][:,QD]
    bus_gs, bus_bs = this_case['bus'][:,GS], this_case['bus'][:,BS]
    # branches
    from_bus = this_case['branch'][:,F_BUS]
    to_bus = this_case['branch'][:,T_BUS]
    br_r = this_case['branch'][:,BR_R]
    br_x = this_case['branch'][:,BR_X]
    br_b = this_case['branch'][:,BR_B]
    tap = this_case['branch'][:,TAP]
    shift = this_case['branch'][:,SHIFT]
    angmin, angmax = this_case['branch'][:,ANGMIN], this_case['branch'][:,ANGMAX]
    flow_lim = this_case['branch'][:,RATE_A]
    branch_list = [((f,t),(t,f),l) for f,t,l in zip(from_bus.astype(int),to_bus.astype(int),range(from_bus.size))] # forward and reversed graph
    # generators
    gen_to_bus = this_case['gen'][:,GEN_BUS]
    gen_mbase = this_case['gen'][:,MBASE]
    gen_pmax, gen_pmin = this_case['gen'][:,PMAX], this_case['gen'][:,PMIN]
    gen_qmax, gen_qmin = this_case['gen'][:,QMAX], this_case['gen'][:,QMIN]
    
    # PRE-PROCESSING DATA
    # normalization to pu
    bus_pd, bus_qd, bus_gs, bus_bs, gen_pmax, gen_qmax, gen_pmin, gen_qmin, flow_lim = \
    bus_pd/baseMVA, bus_qd/baseMVA, bus_gs/baseMVA, bus_bs/baseMVA, gen_pmax/baseMVA, gen_qmax/baseMVA, gen_pmin/baseMVA, gen_qmin/baseMVA, flow_lim/baseMVA
    # insert 1 in tap ratios
    tap = np.where(np.abs(tap)<1e-5,1.,tap)
    # convert angle limits to radians
    angmin, angmax = np.radians(angmin), np.radians(angmax)
    # convert 0 flow lims to large number
    flow_lim = np.where(np.abs(flow_lim)<1e-5,10000.,flow_lim)
    
    # LAMBDA HELPER FUNCTIONS
    gen_on_bus = lambda i: np.where(gen_to_bus == i)[0] # return list og gens attached to bus
    out_lines = lambda i: np.array([w[2] if w[0][0]==i else np.inf for w in branch_list])[np.array([w[2] if w[0][0]==i else np.inf for w in branch_list])!=np.inf].astype(int)
    in_lines = lambda i: np.array([w[2] if w[0][1]==i else np.inf for w in branch_list])[np.array([w[2] if w[0][1]==i else np.inf for w in branch_list])!=np.inf].astype(int)
    
    # NUMBER OF DIFFERENT ELEMENTS
    n_bus = len(bus_list)
    n_branch = len(branch_list)
    n_gen = len(gen_to_bus)
    
    # CVXPY VARIABLES
    Sf = cp.Variable(n_branch,complex=True) # p.u.
    St = cp.Variable(n_branch,complex=True) # p.u.
    Sgen = cp.Variable(n_gen,complex=True) # p.u.
    U = cp.Variable(n_bus,nonneg=True) # p.u.
    W = cp.Variable(n_branch,complex=True) # p.u.
    
    # CVXPY CONSTRAINTS
    constr = []
    # bus constraints (non-balance)
    for busid in bus_list:
        if not DISABLE_VLIMS:
            constr.append(U[busid] <= bus_vmax[busid]**2)
            constr.append(U[busid] >= bus_vmin[busid]**2)
    # branch constraints
    for br in branch_list:
        brf, brr, brid = br
        yl, b = 1 / (br_r[brid] + 1j * br_x[brid]), br_b[brid]
        tl = tap[brid] * (math.cos(shift[brid]) + 1j * math.sin(shift[brid]))
        yff, ytt, yft, ytf = (yl + 1j * (b/2)) / (tl*tl.conjugate()) , (yl + 1j * (b/2)), - yl / tl.conjugate(), -yl / tl
        if not DISABLE_MODEL:
            # forward and reverse flows
            frm, to = brf[0], brf[1]
            constr.append(Sf[brid] == yff.conjugate()*U[frm] + yft.conjugate()*W[brid])
            constr.append(St[brid] == ytt.conjugate()*U[to] + ytf.conjugate()*cp.conj(W[brid]))
        if not DISABLE_FLOWLIMS:
            # flow limits
            constr.append(cp.abs(Sf) <= flow_lim[brid])
            constr.append(cp.abs(St) <= flow_lim[brid])
        if not DISABLE_ANG:
            # angle limits
            constr.append(cp.imag(W[brid]) <= angmax[brid] * cp.real(W[brid]))
            constr.append(cp.imag(W[brid]) >= angmin[brid] * cp.real(W[brid]))
        if not DISABLE_SOCR:
            # second order cones
            frm, to = brf[0], brf[1]
            constr.append(cp.norm(cp.hstack([U[frm]-U[to],2*cp.real(W[brid]),2*cp.imag(W[brid])]),2) <= U[frm]+U[to])
    # generator constraints
    if not DISABLE_GENLIMS:
        constr.append(cp.real(Sgen) <= gen_pmax)
        constr.append(cp.real(Sgen) >= gen_pmin)
        constr.append(cp.imag(Sgen) <= gen_qmax)
        constr.append(cp.imag(Sgen) >= gen_qmin)
    else:
        constr.append(cp.real(Sgen) >= gen_pmin)
    # bus constraints (balance)
    for busid in bus_list:
        Sd = bus_pd[busid] + 1j * bus_qd[busid]
        gens = gen_on_bus(busid)
        Sgen_total = cp.sum([Sgen[gid] for gid in gens]) if gens.size > 0 else 0
        Sf_total = cp.sum([Sf[brid] for brid in out_lines(busid)]) if out_lines(busid).size >0 else 0
        St_total = cp.sum([St[brid] for brid in in_lines(busid)]) if in_lines(busid).size >0 else 0
        Ys = bus_bs[busid] + 1j * bus_gs[busid]
        if not DISABLE_BALANCE:
            constr.append(Sgen_total - Sd - Ys*U[busid] == Sf_total + St_total)
    obj = cp.Minimize(cp.sum(U))
    prob = cp.Problem(obj,constr)
    # solve problem and record results
    try:
        prob.solve(solver='MOSEK',verbose=False)
        if prob.status not in ['infeasible','unbounded']:
            print(f"{cn} solved!\n")
            solved += 1
        else:
            print(f"{cn} not solved!\n")
            unsolved += 1
    except:
        print(f"{cn} encountered solver error!\n")
        unsolved += 1

# DISPLAY OUTCOME
print(f"Solved {100*solved/(solved+unsolved)} percent of cases.")

Status of constraints:
        DISABLE_MODEL: False
        DISABLE_FLOWLIMS: True
        DISABLE_ANG: False
        DISABLE_SOCR: False
        DISABLE_BALANCE: False
        DISABLE_VLIMS: False
        DISABLE_GENLIMS: False
        


0it [00:00, ?it/s]

1it [00:02,  2.31s/it]

pglib_opf_case60_c solved!



2it [00:03,  1.65s/it]

...

pglib_opf_case162_ieee_dtc solved!



21it [02:52, 24.14s/it]

pglib_opf_case1354_pegase solved!



22it [03:21,  9.18s/it]

pglib_opf_case793_goc solved!

Solved 81.81818181818181 percent of cases.
